<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Turkish/sub/tcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turkish Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install focal-loss

In [3]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Turkish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Turkish


In [4]:
ls

cc.tr.300.vec     offenseval-annotation.txt     offenseval-tr-training-v1.tsv
cc.tr.300.vec.gz  offenseval-tr-testset-v1.tsv  readme-trainingset-tr.txt


In [0]:
import pandas as pd
import csv
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train=pd.read_csv( 'offenseval-tr-training-v1.tsv',sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')
train=train.sample(frac=1)

In [6]:
train.head()

,id,tweet,subtask_a
13744,40119,@USER Ne o totoş sağda solda yakalarlar benimd...,OFF
24906,43432,@USER LetGo'ya koyup satalım bari napalım isra...,NOT
15037,23541,Saat 5 buçuk olmuş siyasi yazıyosunuz ya,NOT
28476,37573,Allaha şükür karakterimiz bozuk tipimiz değil,OFF
27440,14411,@USER ben haziran diye gördüm yarın mı cidden ?,NOT


In [0]:
train["tweet"]= train["tweet"].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('URL', '')
train["tweet"]= train["tweet"].str.lower()


In [0]:
train_labels=train['subtask_a']
train=train['tweet']
labels=le.fit_transform(train_labels)

In [9]:
train.tail()

7444      yıllık tecrübesiyle ahirete sizi artık rahat ...
9123       yemin ederim bu sene bejwke kadar kollansayd...
15270          din büyükler için uydurulmuş bir masaldır !
18123      valla bir arkadaşım aradı italyadan tam ital...
8154            yarışalım baba haydi takip edin arkadaşlar
Name: tweet, dtype: object

In [10]:
labels[0:5]

array([1, 0, 0, 1, 0])

In [11]:
import collections
collections.Counter(labels)

Counter({0: 25625, 1: 6131})

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [111]:
len(sequences_matrix),len(labels)

(31756, 31756)

In [0]:
from keras.models import Sequential,Model
from keras.layers import Embedding,CuDNNGRU,Input,Dense,Dropout,Bidirectional,GlobalMaxPooling1D,GlobalAveragePooling1D, concatenate, Concatenate
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import numpy as np
from sklearn.model_selection import StratifiedKFold
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [113]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz

In [0]:
# import gzip
# import shutil
# with gzip.open('cc.tr.300.vec.gz', 'rb') as f_in:
#     with open('cc.tr.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.tr.300.vec', word_index)


In [118]:
embeddings.shape

(10001, 300)

In [0]:
from keras.optimizers import Adam
opt=Adam(0.0001)

In [0]:

# from https://github.com/philipperemy/keras-tcn


import keras.backend as K
import keras.layers
from keras import optimizers
from keras.engine.topology import Layer
from keras.layers import Activation, Lambda
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from typing import List, Tuple


def channel_normalization(x):
    # type: (Layer) -> Layer
    """ Normalize a layer to the maximum activation
    This keeps a layers values between zero and one.
    It helps with relu's unbounded activation
    Args:
        x: The layer to normalize
    Returns:
        A maximal normalized layer
    """
    max_values = K.max(K.abs(x), 2, keepdims=True) + 1e-5
    out = x / max_values
    return out


def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet
    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    Args:
        x: The layer we want to apply the activation to
    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sigm_out])


def residual_block(x, s, i, activation, nb_filters, kernel_size, padding, dropout_rate=0, name=''):
    # type: (Layer, int, int, str, int, int, float, str) -> Tuple[Layer, Layer]
    """Defines the residual block for the WaveNet TCN
    Args:
        x: The previous layer in the model
        s: The stack index i.e. which stack in the overall TCN
        i: The dilation power of 2 we are using for this residual block
        activation: The name of the type of activation to use
        nb_filters: The number of convolutional filters to use in this block
        kernel_size: The size of the convolutional kernel
        padding: The padding used in the convolutional layers, 'same' or 'causal'.
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
        name: Name of the model. Useful when having multiple TCN.
    Returns:
        A tuple where the first element is the residual model layer, and the second
        is the skip connection.
    """

    original_x = x
    conv = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding=padding,
                  name=name + '_dilated_conv_%d_tanh_s%d' % (i, s))(x)
    if activation == 'norm_relu':
        x = Activation('relu')(conv)
        x = Lambda(channel_normalization)(x)
    elif activation == 'wavenet':
        x = wave_net_activation(conv)
    else:
        x = Activation(activation)(conv)

    x = SpatialDropout1D(dropout_rate, name=name + '_spatial_dropout1d_%d_s%d_%f' % (i, s, dropout_rate))(x)

    # 1x1 conv.
    x = Convolution1D(nb_filters, 1, padding='same')(x)
    res_x = keras.layers.add([original_x, x])
    return res_x, x


def process_dilations(dilations):
    def is_power_of_two(num):
        return num != 0 and ((num & (num - 1)) == 0)

    if all([is_power_of_two(i) for i in dilations]):
        return dilations

    else:
        new_dilations = [2 ** i for i in dilations]
        # print(f'Updated dilations from {dilations} to {new_dilations} because of backwards compatibility.')
        return new_dilations


class TCN:
    """Creates a TCN layer.
        Args:
            input_layer: A tensor of shape (batch_size, timesteps, input_dim).
            nb_filters: The number of filters to use in the convolutional layers.
            kernel_size: The size of the kernel to use in each convolutional layer.
            dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
            nb_stacks : The number of stacks of residual blocks to use.
            activation: The activations to use (norm_relu, wavenet, relu...).
            padding: The padding to use in the convolutional layers, 'causal' or 'same'.
            use_skip_connections: Boolean. If we want to add skip connections from input to each residual block.
            return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
            name: Name of the model. Useful when having multiple TCN.
        Returns:
            A TCN layer.
        """

    def __init__(self,
                 nb_filters=64,
                 kernel_size=2,
                 nb_stacks=1,
                 dilations=None,
                 activation='norm_relu',
                 padding='causal',
                 use_skip_connections=True,
                 dropout_rate=0.0,
                 return_sequences=True,
                 name='tcn'):
        self.name = name
        self.return_sequences = return_sequences
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.activation = activation
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        self.padding = padding

        # backwards incompatibility warning.
        # o = tcn.TCN(i, return_sequences=False) =>
        # o = tcn.TCN(return_sequences=False)(i)

        if padding != 'causal' and padding != 'same':
            raise ValueError("Only 'causal' or 'same' paddings are compatible for this layer.")

        if not isinstance(nb_filters, int):
            print('An interface change occurred after the version 2.1.2.')
            print('Before: tcn.TCN(i, return_sequences=False, ...)')
            print('Now should be: tcn.TCN(return_sequences=False, ...)(i)')
            print('Second solution is to pip install keras-tcn==2.1.2 to downgrade.')
            raise Exception()

    def __call__(self, inputs):
        if self.dilations is None:
            self.dilations = [1, 2, 4, 8, 16, 32]
        x = inputs
        x = Convolution1D(self.nb_filters, 1, padding=self.padding, name=self.name + '_initial_conv')(x)
        skip_connections = []
        for s in range(self.nb_stacks):
            for i in self.dilations:
                x, skip_out = residual_block(x, s, i, self.activation, self.nb_filters,
                                             self.kernel_size, self.padding, self.dropout_rate, name=self.name)
                skip_connections.append(skip_out)
        if self.use_skip_connections:
            x = keras.layers.add(skip_connections)
        x = Activation('relu')(x)

        if not self.return_sequences:
            output_slice_index = -1
            x = Lambda(lambda tt: tt[:, output_slice_index, :])(x)
        return x

In [0]:
def model_tcn(embedding_matrix):
    
    inp = Input(shape=(max_len,))
    x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = 'wavenet',name = 'tnc1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = 'wavenet',name = 'tnc2')(x)

    #x = TCN(32,dilations = [1, 2, 4], return_sequences=True, activation = 'wavenet',name = 'tnc3')(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPool1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dense(32, activation="relu")(conc)

    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    
    opt=Adam(0.0001)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss=BinaryFocalLoss(gamma=3), optimizer=opt, metrics=['accuracy'])
    
    return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [124]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.15,stratify=labels,shuffle=True)
model=model_tcn(embeddings)
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val))

Train on 26992 samples, validate on 4764 samples
Epoch 1/10
 - 9s - loss: 0.0616 - acc: 0.8157 - val_loss: 0.0515 - val_acc: 0.8453
Epoch 2/10
 - 6s - loss: 0.0528 - acc: 0.8475 - val_loss: 0.0502 - val_acc: 0.8587
Epoch 3/10
 - 6s - loss: 0.0507 - acc: 0.8531 - val_loss: 0.0487 - val_acc: 0.8585
Epoch 4/10
 - 6s - loss: 0.0496 - acc: 0.8561 - val_loss: 0.0484 - val_acc: 0.8589
Epoch 5/10
 - 6s - loss: 0.0485 - acc: 0.8608 - val_loss: 0.0482 - val_acc: 0.8615
Epoch 6/10
 - 6s - loss: 0.0474 - acc: 0.8632 - val_loss: 0.0498 - val_acc: 0.8520
Epoch 7/10
 - 6s - loss: 0.0462 - acc: 0.8667 - val_loss: 0.0488 - val_acc: 0.8625
Epoch 8/10
 - 6s - loss: 0.0454 - acc: 0.8693 - val_loss: 0.0489 - val_acc: 0.8518
Epoch 9/10
 - 6s - loss: 0.0445 - acc: 0.8734 - val_loss: 0.0484 - val_acc: 0.8571
Epoch 10/10
 - 6s - loss: 0.0426 - acc: 0.8780 - val_loss: 0.0494 - val_acc: 0.8585


In [132]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_val, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.50)

print(classification_report(y_val, y_pred))

4764/4764 [==============================] - 0s 82us/step
              precision    recall  f1-score   support

           0       0.88      0.95      0.92      3844
           1       0.70      0.47      0.56       920

    accuracy                           0.86      4764
   macro avg       0.79      0.71      0.74      4764
weighted avg       0.85      0.86      0.85      4764



In [133]:
test=pd.read_csv( 'offenseval-tr-testset-v1.tsv',sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')
test["tweet"]= test["tweet"].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('URL', '')
test['tweet']=test['tweet'].str.lower()

test.head()

,id,tweet
0,41993,sayın başkanım bu şekilde devam inşallah👏
1,23000,"herkes gevşekliği kadar duyar kasıyor,hayat bö..."
2,42478,olgun ilişkisi olan arkadaş size en güzel hedi...
3,21748,burada atıp tutacağına o kötü koşullarda ku...
4,13607,i̇şte o onur dediğin sende yok sorun o işte


In [0]:
X_test=test['tweet']

test_sequences = tokenize.texts_to_sequences(X_test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [135]:
y_test = model.predict(X_test_sequences, batch_size=64, verbose=1)


3528/3528 [==============================] - 0s 79us/step


In [0]:
np.save('/content/tcn.npy',y_test)

In [0]:
y_test[y_test>0.45]=1
y_test[y_test<0.45]=0
y_test=y_test.astype('int16').ravel()

In [74]:
y_test=le.inverse_transform(y_test)
y_test=pd.DataFrame(y_test,columns=['label'])
y_test=pd.concat([test['id'], y_test['label']], axis=1)
y_test.head()


,id,label
0,41993,NOT
1,23000,NOT
2,42478,NOT
3,21748,NOT
4,13607,NOT


In [0]:
y_test.to_csv('/content/sub1.csv',index=False,header=None)

In [0]:
import keras
keras.backend.clear_session()

In [0]:
import tensorflow as tf
tf.reset_default_graph()